In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Sequential

In [12]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
diabetes = load_diabetes()

data = diabetes['data']
targets = diabetes['target']

In [13]:
X_train,X_val,y_train,y_val = train_test_split(data, targets, test_size = 0.1)

#### Dummy Model

In [14]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  
])

In [15]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

### Custom Callback
##### we will use logs dictionary to access the loss and metric value

In [16]:
class customCallback(tf.keras.callbacks.Callback):
    
    def on_train_batch_end(self,batch,logs = None):
        if batch % 2 is 0:
            print( f"\n After Batch {batch}, loss is {logs['loss']}" )

    def on_test_batch_end(self,batch,logs=None):
        if batch % 2 is 0:
            print(f"\n After Batch {batch}, loss is {logs['loss']} ")

    def on_epoch_end(self, epoch, logs = None):
        print(f"Epoch {epoch}, Mean Absolute Error is {logs['mae']}, Loss is {logs['loss']}")
        
    def on_predict_batch_end(self, batch, logs=None):
        print(f"Finished Prediction on Batch {batch}")

In [17]:
history = model.fit(X_train, y_train, epochs=10, callbacks=[customCallback()], verbose = 0, batch_size=2**6)


 After Batch 0, loss is 23378.5390625

 After Batch 2, loss is 26356.875

 After Batch 4, loss is 28064.83984375

 After Batch 6, loss is 28902.17578125
Epoch 0, Mean Absolute Error is 151.17138671875, Loss is 28902.17578125

 After Batch 0, loss is 29013.181640625

 After Batch 2, loss is 27027.478515625

 After Batch 4, loss is 26629.837890625

 After Batch 6, loss is 28592.13671875
Epoch 1, Mean Absolute Error is 150.22731018066406, Loss is 28592.13671875

 After Batch 0, loss is 32774.3359375

 After Batch 2, loss is 29210.560546875

 After Batch 4, loss is 28264.087890625

 After Batch 6, loss is 28034.705078125
Epoch 2, Mean Absolute Error is 148.52096557617188, Loss is 28034.705078125

 After Batch 0, loss is 24783.283203125

 After Batch 2, loss is 25610.365234375

 After Batch 4, loss is 26349.353515625

 After Batch 6, loss is 27056.611328125
Epoch 3, Mean Absolute Error is 145.59791564941406, Loss is 27056.611328125

 After Batch 0, loss is 23100.11328125

 After Batch 2, l

In [18]:
model.evaluate(X_val,y_val, callbacks=[customCallback()], verbose=0, batch_size=10)


 After Batch 0, loss is 19756.912109375 

 After Batch 2, loss is 20266.62890625 

 After Batch 4, loss is 23119.00390625 


[23119.00390625, 136.37364196777344]

In [19]:
model.predict(X_val,batch_size=10, callbacks=[customCallback()], verbose=False )

Finished Prediction on Batch 0
Finished Prediction on Batch 1
Finished Prediction on Batch 2
Finished Prediction on Batch 3
Finished Prediction on Batch 4


array([[ 8.376762 ],
       [12.174041 ],
       [22.290215 ],
       [22.776358 ],
       [10.463553 ],
       [13.194703 ],
       [38.789448 ],
       [18.788671 ],
       [19.576712 ],
       [20.279705 ],
       [37.07193  ],
       [22.324568 ],
       [10.980113 ],
       [16.393162 ],
       [24.081112 ],
       [32.423786 ],
       [ 9.131023 ],
       [32.69382  ],
       [34.415005 ],
       [21.46859  ],
       [35.131714 ],
       [ 9.228954 ],
       [ 9.9706335],
       [24.063854 ],
       [18.500978 ],
       [27.93496  ],
       [ 8.815715 ],
       [18.43633  ],
       [26.393787 ],
       [18.940454 ],
       [16.778429 ],
       [39.354088 ],
       [26.142286 ],
       [27.714622 ],
       [10.443917 ],
       [22.157833 ],
       [32.663357 ],
       [17.78948  ],
       [30.428757 ],
       [33.36573  ],
       [13.128178 ],
       [10.773067 ],
       [ 8.69389  ],
       [33.32542  ],
       [25.316082 ]], dtype=float32)

### We will define a custom callback to reduce the learning rate w.r.t to # of Epochs

##### It is going to have a more complex custom callback

In [20]:
lr_schedule = [
    (5,0.05), (10,0.03), (15,0.02), (20,0.01)
]
# we will get new learning rate using this function by comparing to list above.
def get_new_learning_rate(epoch, lr):
    for i in lr_schedule:
        if epoch in i:
            lr = i[1]

    return lr

In [44]:
class Learning_rate_scheduler( tf.keras.callbacks.Callback ):
     def __init__(self, new_lr):
        super(Learning_rate_scheduler, self).__init__
        #adding new learning rate function to our callback
        self.new_lr = new_lr
    
     def on_epoch_begin(self, epoch, logs=None):
        #we will check if our optimizer has learning rate option or not
        try:
            curr_rate = tf.keras.backend.get_value(self.model.optimizer.lr)

            #calling auxillary function to get scheduled learning rate, we have passed the function as parameter which is new_lr

            scheduled_rate = self.new_lr(epoch, curr_rate)

            tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)

            print(f"Learning Rate for Epoch {epoch} is {tf.keras.backend.get_value(self.model.optimizer.lr)}")

        except Exception as E:
            print(f'{E}\n Most Probably your optimizer do not have learing rate option.')

In [45]:
model = Sequential([
    
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [46]:
model.compile(loss='mse',
                optimizer="adam",
                metrics=['mae', 'mse'])

In [47]:
model.fit(X_train, y_train, epochs=25, batch_size=64, callbacks=[Learning_rate_scheduler(get_new_learning_rate)], verbose=False)

Learning Rate for Epoch 0 is 0.0010000000474974513
Learning Rate for Epoch 1 is 0.0010000000474974513
Learning Rate for Epoch 2 is 0.0010000000474974513
Learning Rate for Epoch 3 is 0.0010000000474974513
Learning Rate for Epoch 4 is 0.0010000000474974513
Learning Rate for Epoch 5 is 0.05000000074505806
Learning Rate for Epoch 6 is 0.05000000074505806
Learning Rate for Epoch 7 is 0.05000000074505806
Learning Rate for Epoch 8 is 0.05000000074505806
Learning Rate for Epoch 9 is 0.05000000074505806
Learning Rate for Epoch 10 is 0.029999999329447746
Learning Rate for Epoch 11 is 0.029999999329447746
Learning Rate for Epoch 12 is 0.029999999329447746
Learning Rate for Epoch 13 is 0.029999999329447746
Learning Rate for Epoch 14 is 0.029999999329447746
Learning Rate for Epoch 15 is 0.019999999552965164
Learning Rate for Epoch 16 is 0.019999999552965164
Learning Rate for Epoch 17 is 0.019999999552965164
Learning Rate for Epoch 18 is 0.019999999552965164
Learning Rate for Epoch 19 is 0.019999999